# Data Science Fundamentals - Assignment 3
##### By Alexandra de Carvalho, nmec 93346

This work aims at predicting stock price movements with the use of a machine learning model. 

For this purpose, this work also focuses on processing, exploring and visualizing the dataset, comprised of both numerical historical stock price records and tweet posts on stock price movements, collected during many time points over a two year period.

In [35]:
import pandas as pd
import os   

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)
pd.set_option('display.max_colwidth', 100)

## 1 - Dataset Preprocessing

### Loading the datasets

Preprocessing is a key step in any analysis and can affect how good the results will be. The dataset provided is divided into price information and tweet records folders. Furthermore, each modality is divided by company, according to the table below.

In [42]:
stockTable = pd.read_table('data/StockTable')
stockTable

,Sector,Symbol,Company
0,Basic Matierials,$XOM,Exxon Mobil Corporation
1,Basic Matierials,$RDS-B,Royal Dutch Shell
2,Basic Matierials,$PTR,PetroChina Company Limited
3,Basic Matierials,$CVX,Chevron Corporation
4,Basic Matierials,$TOT,TOTAL S.A.
5,Basic Matierials,$BP,BP p.l.c.
6,Basic Matierials,$BHP,BHP Billiton Limited
7,Basic Matierials,$SNP,China Petroleum & Corporation
8,Basic Matierials,$SLB,Schlumberger Limited
9,Basic Matierials,$BBL,BHP Billiton plc


Let's start by reading the price records for all companies and concatenating them into a single dataframe. For this to happen, we will go through each .CSV file in the 'data/price/raw' folder and load each record file into a dataframe. Since the name of the company each file is about is important information that for now is only contained in the name of the file, let's add that as a column. Then, as we go through all files, we keep concatenating them together into one big final dataframe.   

In [38]:
def list_files(dir):                                                                                                  
    r = []                                                                                                            
    subdirs = [x[0] for x in os.walk(dir)]                                                                            
    for subdir in subdirs:                                                                                            
        files = os.walk(subdir).__next__()[2]                                                                             
        if (len(files) > 0):                                                                                          
            for file in files:                                                                                        
                r.append(os.path.join(subdir, file))                                                                         
    return r 

In [40]:
path = r'data/price/raw'
allFiles = list_files(path)

stockPrices = pd.DataFrame(columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Company'])

for filename in allFiles:
    company = filename.split(".")[0].split("/")[-1] 
    newDf = pd.read_csv(filename, index_col=None, header=0)
    newDf['Company'] = [company]*newDf.shape[0]
    stockPrices = pd.concat([stockPrices,newDf],axis=0)

stockPrices

,Date,Open,High,Low,Close,Adj Close,Volume,Company
0,2012-09-04,20.629999,20.690001,20.480000,20.510000,17.401751,38173600,GE
1,2012-09-05,20.549999,20.770000,20.510000,20.650000,17.520535,33033500,GE
2,2012-09-06,20.870001,21.320000,20.850000,21.309999,18.080509,53178600,GE
3,2012-09-07,21.320000,21.590000,21.299999,21.590000,18.318081,36379800,GE
4,2012-09-10,21.480000,21.650000,21.430000,21.480000,18.224752,33672300,GE
...,...,...,...,...,...,...,...,...
1253,2017-08-28,102.489998,102.849998,102.080002,102.559998,102.559998,5630400,DIS
1254,2017-08-29,102.000000,102.989998,101.820000,102.570000,102.570000,6011700,DIS
1255,2017-08-30,102.480003,103.449997,102.480003,102.870003,102.870003,6573200,DIS
1256,2017-08-31,103.000000,103.190002,101.000000,101.199997,101.199997,9823800,DIS


Besides the added 'company' column, the previous dataframe has a 'date' column, an 'open' column with the company's price at the stock opening time of that day and a 'close' column with the company's price at the stock closing time of that day. There is also 'high' and 'low' columns, with the highest and lowest prices, respectively, at which the company's stock traded during that day. The 'adj close' column refers to the the adjusted close price, which is the final price of a company's stock for that day, after adjustments for all applicable splits and dividend distributions are applied. At last, the 'volume' column measures the number of shares that were traded that day. We can see we have 108592 records, over 87 companies and two years.

Now, we should do the same as above for the tweets. In the raw tweets folder, each company has multiple files, divided by timestamp (day), so we will need one more step along the way.

In [50]:
path = r'data/tweet/raw'
allFiles = list_files(path)

stockTweets = pd.DataFrame(columns = ['created_at', 'id', 'id_str', 'text', 'source', 'truncated', 'in_reply_to_status_id', 
'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 
'coordinates', 'place', 'contributors', 'retweeted_status', 'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted', 
'filter_level', 'lang', 'possibly_sensitive', 'company'])

for filename in allFiles:
    company = filename.split("/")[-2] 
    newDf = pd.read_json(filename, lines=True)
    newDf['company'] = [company]*newDf.shape[0]
    stockTweets = pd.concat([stockTweets,newDf],axis=0)

stockTweets

                 created_at                  id              id_str  \
0 2014-07-10 18:15:00+00:00  487298961544212480  487298961544212480   
1 2014-07-09 22:14:31+00:00  486996850021773312  486996850021773312   

                                                                         text  \
0   Is it breakout time on $VFC $SO $ERIE $IGN #NASDAQ http://t.co/7GThIfnq4C   
1  $SO Buying  or selling? $NEE $LPNT $CHTP #financial http://t.co/ktG57MeKys   

                                                                    source  truncated  \
0  <a href="https://mobile.twitter.com" rel="nofollow">Mobile Web (M2)</a>      False   
1  <a href="https://mobile.twitter.com" rel="nofollow">Mobile Web (M2)</a>      False   

   in_reply_to_status_id  in_reply_to_status_id_str  in_reply_to_user_id  in_reply_to_user_id_str  \
0                    NaN                        NaN                  NaN                      NaN   
1                    NaN                        NaN                  NaN

KeyboardInterrupt: 

In this case, feature selection or dimensionality reduction techniques are not needed, as the columns presented are exactly the ones we need.

With our datasets ready, let's get to work! Firstly, it is useful to investigate if there are any missing values, and, if so, decide how to deal with, based on a quick analysis of the root of the missing values.

In the test dataframe, there are no missing values. Because the missing values on the training dataframe are on text reviews, they will affect our work. Since there are only 10 missing values out of 3000+ rows, removing those rows won't affect the reliability of our analysis. So, let's drop those records, as well as duplicated records.